In [1]:
import numpy as np
import skimage
import utils
import pathlib

C:\Users\Marco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [168]:
def otsu_thresholding(im: np.ndarray) -> int:


    """
    Otsu's thresholding algorithm that segments an image into 1 or 0 (True or False)
    The function takes in a grayscale image and outputs a threshold value

    args:
        im: np.ndarray of shape (H, W) in the range [0, 255] (dtype=np.uint8)
    return:
        (int) the computed thresholding value
    """

    assert im.dtype == np.uint8

    #normalized histogram
    hist = np.histogram(im, bins=np.arange(257), density=True)[0]
   
    #list for cumulative sums
    cumulative_sums = []
    #counter for cumulative sums
    current_sum = 0 
    comp_cum= []
    
    #cumulative sum--->histogram
    for i in hist:
        current_sum+=i
        cumulative_sums.append(current_sum)
        comp_cum.append(1-current_sum)
       
    
    #convert to np array to iterate easily
    cumulative_sums_np = np.array(cumulative_sums)
    comp_cum_np = np.array(comp_cum)
    
    #cumulative average using numpy
    cumulative_mean = np.cumsum(np.arange(256) * hist)
    mg = cumulative_mean[len(cumulative_mean)-1] #global mean

    if np.not_equal(cumulative_sums_np*comp_cum_np,0.00000000e+00).any(): #to avoid division by 0
        sig = (mg * cumulative_sums_np - cumulative_mean)**2 / (cumulative_sums_np * comp_cum_np)
     
    sig =np.nan_to_num(sig, copy=True, nan=0.0) #replace generated nan values
    
    return np.argmax((sig))

In [169]:
if __name__ == "__main__":
    # DO NOT CHANGE
    impaths_to_segment = [
        pathlib.Path("thumbprint.png"),
        pathlib.Path("polymercell.png")
    ]
    for impath in impaths_to_segment:
        im = utils.read_image(impath)
        threshold = otsu_thresholding(im)
        print("Found optimal threshold:", threshold)

        # Segment the image by threshold
        segmented_image = (im >= threshold)
        assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
                im.shape, segmented_image.shape)
        assert segmented_image.dtype == np.bool, "Expected thresholded image dtype to be np.bool. Was: {}".format(
                segmented_image.dtype)

        segmented_image = utils.to_uint8(segmented_image)

        save_path = "{}-segmented.png".format(impath.stem)
        utils.save_im(save_path, segmented_image)

Reading image: images\thumbprint.png
Found optimal threshold: 153
Saving image to: image_processed\thumbprint-segmented.png
Reading image: images\polymercell.png
Found optimal threshold: 181
Saving image to: image_processed\polymercell-segmented.png


C:\Users\Marco\AppData\Local\Temp\ipykernel_14860\2867839640.py:41: RuntimeWarning: invalid value encountered in true_divide
  sig = (mg * cumulative_sums_np - cumulative_mean)**2 / (cumulative_sums_np * comp_cum_np)
C:\Users\Marco\AppData\Local\Temp\ipykernel_14860\1997791817.py:16: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  assert segmented_image.dtype == np.bool, "Expected thresholded image dtype to be np.bool. Was: {}".format(
